In [1]:
# Import necessary libraries
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from missforest import MissForest
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Initialize FaceNet
embedder = FaceNet()

# Function to get embeddings
def get_embedding(face_img):
    face_img = face_img.astype('float32')  # 3D tensor (160 x 160 x 3)
    face_img = np.expand_dims(face_img, axis=0)  # 4D tensor for FaceNet
    yhat = embedder.embeddings(face_img)
    return yhat[0]  # 512D embedding

# Define the FACELOADING class with data augmentation
class FACELOADING:

    def __init__(self, directory: str, augment=False, augment_count=5):
        self.directory = directory
        self.target_size = (160, 160)
        self.detector = MTCNN()
        self.augment = augment
        self.augment_count = augment_count
        if self.augment:
            self.datagen = ImageDataGenerator(
                rotation_range=30,
                horizontal_flip=True,
                shear_range=0.2,
                brightness_range=(0.7, 1.3),
                zoom_range=0.2,
                width_shift_range=0.2,
                height_shift_range=0.2
            )

    # Handle every image
    def extract_face(self, filename: str) -> np.ndarray:
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        detections = self.detector.detect_faces(img)
        if len(detections) == 0:
            return None
        x, y, w, h = detections[0]['box']
        x, y = abs(x), abs(y)  # Ensure positive values
        face = img[y:y+h, x:x+w]
        face_arr = cv.resize(face, self.target_size)
        return face_arr

    def augment_face(self, face_img, augment_count):
        face_img = face_img.reshape((1,) + face_img.shape)  # Reshape for ImageDataGenerator
        augmented_faces = []
        i = 0
        for batch in self.datagen.flow(face_img, batch_size=1, seed=42):
            augmented_face = batch[0].astype('uint8')
            augmented_faces.append(augmented_face)
            i += 1
            if i >= augment_count:
                break
        return augmented_faces

    def load_data(self):
        X_data = []
        Y_data = []
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)
            if not os.path.isdir(path):
                continue
            for im_name in os.listdir(path):
                try:
                    img_path = os.path.join(path, im_name)
                    single_face = self.extract_face(filename=img_path)
                    if single_face is not None:
                        if self.augment:
                            # Generate augmented images
                            augmented_faces = self.augment_face(single_face, self.augment_count)
                            X_data.extend(augmented_faces)
                            Y_data.extend([sub_dir] * len(augmented_faces))
                        else:
                            X_data.append(single_face)
                            Y_data.append(sub_dir)
                except Exception as e:
                    print(f"Error loading image {im_name}: {e}")
                    pass
        return np.asarray(X_data), np.asarray(Y_data)

    def plot_images(self, images):
        plt.figure(figsize=(16, 12))
        for num, image in enumerate(images[:9]):  # Plot first 9 images
            ncols = 3
            nrows = 3
            plt.subplot(nrows, ncols, num+1)
            plt.imshow(image.astype('uint8'))
            plt.axis("off")
        plt.show()

# Number of augmented images per person
total_augmented_images = 50
train_augmented_images = 40
test_augmented_images = total_augmented_images - train_augmented_images

# Initialize FACELOADING with augmentation for both training and test data
faceloading = FACELOADING("dataset", augment=True, augment_count=total_augmented_images)
X_data, Y_data = faceloading.load_data()

# Encode labels
encoder = LabelEncoder()
encoder.fit(Y_data)
Y_encoded = encoder.transform(Y_data)

# Split augmented images into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train_encoded, Y_test_encoded = train_test_split(
    X_data, Y_encoded, stratify=Y_encoded, test_size=test_augmented_images/total_augmented_images, random_state=42
)

# Generate embeddings for training data
print("Generating embeddings for training data...")
X_train_embeddings = []
for img in tqdm(X_train):
    embedding = get_embedding(img)
    X_train_embeddings.append(embedding)
X_train_embeddings = np.asarray(X_train_embeddings)

# Generate embeddings for testing data
print("Generating embeddings for testing data...")
X_test_embeddings = []
for img in tqdm(X_test):
    embedding = get_embedding(img)
    X_test_embeddings.append(embedding)
X_test_embeddings = np.asarray(X_test_embeddings)




2024-12-25 19:32:54.292461: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-25 19:32:54.357348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 19:32:54.404980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-25 19:32:54.421536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 19:32:54.493909: I tensorflow/core/platform/cpu_feature_guar

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


I0000 00:00:1735137190.463413   27551 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 634ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 738ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


  0%|                                                                                           | 0/400 [00:00<?, ?it/s]

...

 95%|█████████████████████████████████████████████████████████████████████████████▉    | 95/100 [00:04<00:00, 23.73it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 98/100 [00:04<00:00, 23.26it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 20.79it/s]


In [2]:
# Apply synthetic occlusions to test images
def apply_random_occlusion(image, occlusion_rate):
    occluded_image = image.copy()
    h, w, _ = image.shape
    num_pixels = h * w
    num_occlusion = int(occlusion_rate * num_pixels)
    for _ in range(num_occlusion):
        x = np.random.randint(0, w)
        y = np.random.randint(0, h)
        occluded_image[y, x, :] = 0  # Set pixel to black
    return occluded_image

occlusion_rates = [0.3, 0.4]
occluded_X_test = {rate: [] for rate in occlusion_rates}
for rate in occlusion_rates:
    for img in X_test:
        occluded_img = apply_random_occlusion(img, rate)
        occluded_X_test[rate].append(occluded_img)



In [3]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

def impute_image(image, method='mice'):
    h, w, c = image.shape
    # Convert to float32 to allow NaN assignment
    flat_img = image.astype('float32').reshape(-1, c)
    # Create a mask for missing pixels (black pixels)
    mask = np.all(flat_img == [0, 0, 0], axis=1)
    # Replace black pixels with NaN
    flat_img[mask] = np.nan

    if method == 'mice':
        imputer = IterativeImputer(max_iter=10, random_state=0)
    elif method == 'missforest':
        # Use RandomForestRegressor for imputation
        def random_forest_imputer(X):
            model = RandomForestRegressor(n_estimators=100, random_state=0)
            mask = np.isnan(X)
            X_imputed = X.copy()
            for feature in range(X.shape[1]):
                if np.any(mask[:, feature]):  # Only apply to columns with missing data
                    not_missing = ~mask[:, feature]
                    model.fit(X[not_missing], X[not_missing, feature])
                    X_imputed[mask[:, feature], feature] = model.predict(X[mask[:, feature]])
            return X_imputed
        
        flat_img = random_forest_imputer(flat_img)
    else:
        raise ValueError("Invalid imputation method.")

    # Reshape back to the original image dimensions
    imputed_image = flat_img.reshape(h, w, c).astype('uint8')
    return imputed_image



# Impute occluded images
imputed_X_test = {}
for method in ['mice', 'missforest']:
    imputed_X_test[method] = {}
    for rate in occluded_X_test.keys():
        print(f"Imputing with {method}, Occlusion Rate: {int(rate*100)}%")
        imputed_images = []
        for img in tqdm(occluded_X_test[rate]):
            imputed_img = impute_image(img, method=method)
            imputed_images.append(imputed_img)
        imputed_X_test[method][rate] = imputed_images

# Generate embeddings for imputed test data
print("Generating embeddings for imputed test data...")
imputed_X_test_embeddings = {}
for method in imputed_X_test.keys():
    imputed_X_test_embeddings[method] = {}
    for rate in imputed_X_test[method].keys():
        embeddings = []
        print(f"Generating embeddings for {method}, Occlusion Rate: {int(rate*100)}%")
        for img in tqdm(imputed_X_test[method][rate]):
            embedding = get_embedding(img)
            embeddings.append(embedding)
        imputed_X_test_embeddings[method][rate] = np.asarray(embeddings)

# Train SVM classifier
print("Training SVM classifier...")
model = SVC(kernel='linear', probability=True)
model.fit(X_train_embeddings, Y_train_encoded)

# Evaluate the model
print("Evaluating the model...")
for method in imputed_X_test_embeddings.keys():
    for rate in imputed_X_test_embeddings[method].keys():
        embeddings = imputed_X_test_embeddings[method][rate]
        y_pred = model.predict(embeddings)
        accuracy = accuracy_score(Y_test_encoded, y_pred)
        print(f"SVM Accuracy - Imputation Method: {method}, Occlusion Rate: {int(rate*100)}%, Accuracy: {accuracy*100:.2f}%")



Imputing with mice, Occlusion Rate: 30%


100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1239.01it/s]


Imputing with mice, Occlusion Rate: 40%


100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1195.61it/s]


Imputing with missforest, Occlusion Rate: 30%


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


 97%|███████████████████████████████████████████████████████████████████████████████▌  | 97/100 [00:04<00:00, 20.85it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 20.49it/s]

Training SVM classifier...
Evaluating the model...
SVM Accuracy - Imputation Method: mice, Occlusion Rate: 30%, Accuracy: 49.00%
SVM Accuracy - Imputation Method: mice, Occlusion Rate: 40%, Accuracy: 47.00%
SVM Accuracy - Imputation Method: missforest, Occlusion Rate: 30%, Accuracy: 100.00%
SVM Accuracy - Imputation Method: missforest, Occlusion Rate: 40%, Accuracy: 99.00%


In [4]:
# Classification using distance metrics
def classify_using_distance(train_embeddings, train_labels, test_embeddings, metric='euclidean'):
    predictions = []
    if metric == 'euclidean':
        distances = euclidean_distances(test_embeddings, train_embeddings)
    elif metric == 'cityblock':
        distances = manhattan_distances(test_embeddings, train_embeddings)
    else:
        raise ValueError("Invalid metric.")
    min_indices = np.argmin(distances, axis=1)
    predictions = train_labels[min_indices]
    return predictions

# Evaluate using distance metrics
print("Evaluating using distance metrics...")
for method in imputed_X_test_embeddings.keys():
    for rate in imputed_X_test_embeddings[method].keys():
        embeddings = imputed_X_test_embeddings[method][rate]
        # Euclidean distance
        y_pred_euc = classify_using_distance(X_train_embeddings, Y_train_encoded, embeddings, metric='euclidean')
        accuracy_euc = accuracy_score(Y_test_encoded, y_pred_euc)
        print(f"Euclidean - Method: {method}, Occlusion Rate: {int(rate*100)}%, Accuracy: {accuracy_euc*100:.2f}%")
        # City Block distance
        y_pred_cb = classify_using_distance(X_train_embeddings, Y_train_encoded, embeddings, metric='cityblock')
        accuracy_cb = accuracy_score(Y_test_encoded, y_pred_cb)
        print(f"City Block - Method: {method}, Occlusion Rate: {int(rate*100)}%, Accuracy: {accuracy_cb*100:.2f}%")



Evaluating using distance metrics...
Euclidean - Method: mice, Occlusion Rate: 30%, Accuracy: 53.00%
City Block - Method: mice, Occlusion Rate: 30%, Accuracy: 58.00%
Euclidean - Method: mice, Occlusion Rate: 40%, Accuracy: 53.00%
City Block - Method: mice, Occlusion Rate: 40%, Accuracy: 52.00%
Euclidean - Method: missforest, Occlusion Rate: 30%, Accuracy: 100.00%
City Block - Method: missforest, Occlusion Rate: 30%, Accuracy: 100.00%
Euclidean - Method: missforest, Occlusion Rate: 40%, Accuracy: 100.00%
City Block - Method: missforest, Occlusion Rate: 40%, Accuracy: 99.00%


In [5]:
# Save embeddings and labels for use in `main.py`
np.savez_compressed("faces_embeddings_new1.npz", embeddings=X_train_embeddings, labels=Y_train_encoded)

# Save the trained model and label encoder
with open("svm_model_new1.pkl", 'wb') as file:
    pickle.dump(model, file)
with open("label_encoder_new1.pkl", 'wb') as file:
    pickle.dump(encoder, file)


In [6]:
# Save the trained model and embeddings only for "missforest"
print("Saving the trained model and embeddings for 'missforest' imputation method...")

# Choose a specific occlusion rate to save embeddings for
missforest_rate = 0.3  # You can change this to another rate if needed

# Ensure "missforest" embeddings exist for the selected rate
if 'missforest' in imputed_X_test_embeddings and missforest_rate in imputed_X_test_embeddings['missforest']:
    embeddings_to_save = imputed_X_test_embeddings['missforest'][missforest_rate]
    labels_to_save = Y_test_encoded  # Assuming labels are shared across methods
    model_to_save = model  # The SVM model trained earlier
else:
    raise ValueError("'missforest' embeddings are missing or the specified occlusion rate is not found.")

# Save embeddings and labels for use in `main.py`
np.savez_compressed("faces_embeddings_miss1.npz", embeddings=embeddings_to_save, labels=labels_to_save)

# Save the trained SVM model
with open("svm_model_miss1.pkl", 'wb') as file:
    pickle.dump(model_to_save, file)

# Save the label encoder
with open("label_encoder_miss1.pkl", 'wb') as file:
    pickle.dump(encoder, file)

print("Files saved:")
print("- faces_embeddings.npz")
print("- svm_model.pkl")
print("- label_encoder.pkl")


Saving the trained model and embeddings for 'missforest' imputation method...
Files saved:
- faces_embeddings.npz
- svm_model.pkl
- label_encoder.pkl
